In [1]:
import pandas as pd
import os
import re
import multiprocess as mp

In [2]:
def import_file(f):
    
    reviews = pd.DataFrame(columns=['product_title', 'star_rating',
        'helpful_votes', 'total_votes', 'verified_purchase',
        'review_headline', 'review_body', 'category'])
    
    category = re.search('amazon_reviews_us_(.+?)_v1|amazon_reviews_(.+?)_US_v1', f).group(1)
    
    col_list = ['product_title', 'star_rating',
        'helpful_votes', 'total_votes', 'verified_purchase',
        'review_headline', 'review_body']
    df = pd.read_csv("./archive-6/" + str(f), sep='\t', error_bad_lines=False, usecols=col_list)
    
    df['category'] = category
    df = df[df.filter(like='review_').notna().all(1)]
    df.verified_purchase = df.verified_purchase.eq('Y').mul(1)

    reviews = reviews.append(df)
    return(reviews)

In [3]:
num_threads = mp.cpu_count()
pool = mp.Pool(num_threads)

(96832, 8)
(144720, 8)
(380549, 8)


/Users/sarahtorrence/opt/anaconda3/lib/python3.8/site-packages/multiprocess/process.py:108: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)


(1749022, 8)


In [4]:
files = os.listdir('archive-6')
r = pool.map(import_file, files)

In [5]:
reviews = pd.concat(r)

In [6]:
reviews.shape

(2371123, 8)

In [7]:
pool.close()